In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('dataset/rainfall.csv')

In [3]:
df

Year  Month  Max_Temp  Min_Temp  Rainfall  Humidity  Wind_Speed    Lat  \
0      1949      1      29.4      12.3         0      68.0    0.453704  22.70   
1      1950      1      30.0      14.1         0      77.0    0.453704  22.70   
2      1951      1      28.2      12.3         0      77.0    0.453704  22.70   
3      1952      1      26.6      12.3         1      77.0    0.453704  22.70   
4      1953      1      30.0      13.3         2      75.0    0.453704  22.70   
...     ...    ...       ...       ...       ...       ...         ...    ...   
21115  2009     12      30.3      16.5         0      72.0    2.800000  20.87   
21116  2010     12      31.7      16.7         3      79.0    2.400000  20.87   
21117  2011     12      31.5      16.4         0      73.0    0.000000  20.87   
21118  2012     12      30.0      15.8         0      70.0    1.800000  20.87   
21119  2013     12      29.9      16.5         0      72.0    3.000000  20.87   

         Lon  
0      90.36  
1      90.36  
2      90.36  
3      90.36  
4      90.36  
...      ...  
21115  92.26  
21116  92.26  
21117  92.26  
21118  92.26  
21119  92.26  

[21120 rows x 9 columns]

In [4]:
df.isnull().sum()

Year          0
Month         0
Max_Temp      0
Min_Temp      0
Rainfall      0
Humidity      0
Wind_Speed    0
Lat           0
Lon           0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21120 entries, 0 to 21119
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        21120 non-null  int64  
 1   Month       21120 non-null  int64  
 2   Max_Temp    21120 non-null  float64
 3   Min_Temp    21120 non-null  float64
 4   Rainfall    21120 non-null  int64  
 5   Humidity    21120 non-null  float64
 6   Wind_Speed  21120 non-null  float64
 7   Lat         21120 non-null  float64
 8   Lon         21120 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 1.5 MB


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separate features (X) and target variable (y)
X = df.drop(columns=['Rainfall']).values
y = df['Rainfall'].values

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler to training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier


classifiers = [
    DecisionTreeClassifier(max_depth = 10),
    SVC(kernel = 'linear', C = 1,decision_function_shape='ovr'),
    KNeighborsClassifier(n_neighbors = 17),
    GaussianNB(),
    MLPClassifier(hidden_layer_sizes = [100]*5),
    RandomForestClassifier()
]

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Accuracy','F1-Score', 'Precision', 'Recall'])

# Train and evaluate each classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    f1_score = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    
    
    # Append results to DataFrame
    results_df = pd.concat([results_df,pd.DataFrame([{"Model": clf_name,"Accuracy": accuracy,'F1-Score': f1_score, 
                                    'Precision': precision, 'Recall': recall}])],ignore_index=True)
    
results_df

C:\Users\User\AppData\Local\Temp\ipykernel_12796\4236304167.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df,pd.DataFrame([{"Model": clf_name,"Accuracy": accuracy,'F1-Score': f1_score,
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero

Model  Accuracy  F1-Score  Precision    Recall
0  DecisionTreeClassifier  0.791903  0.771086   0.759288  0.791903
1                     SVC  0.778883  0.736801   0.701470  0.778883
2    KNeighborsClassifier  0.785275  0.755282   0.745034  0.785275
3              GaussianNB  0.765152  0.750397   0.756518  0.765152
4           MLPClassifier  0.766335  0.757637   0.750500  0.766335
5  RandomForestClassifier  0.818655  0.794952   0.793862  0.818655

In [9]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
model.fit(X_train_scaled,y_train)
pred=model.predict(X_test_scaled)
pred_prob=model.predict_proba(X_test_scaled)
pred_prob

array([[0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.03, 0.97],
       ...,
       [0.01, 0.  , 0.01, 0.98],
       [0.  , 0.  , 0.  , 1.  ],
       [0.3 , 0.1 , 0.49, 0.11]])

In [10]:
#import pickle
#pickle.dump(model,open("model.pkl",'wb'))
#pickle.dump(scaler,open("scaler.pkl",'wb'))